## Accessing NOAA U.S. Gridded Climate Normals with the Planetary Computer STAC API

The [NOAA U.S. Gridded Climate Normals](https://www.ncei.noaa.gov/products/land-based-station/us-climate-normals#tab-1027) provide gridded temperature and precipitation datasets across the contiguous United States (CONUS) at approximately 5km resolution. The gridded normals include annual, seasonal, monthly and daily long-term averages computed from NOAA's NClimGrid dataset. Normals act both as a ruler to compare current weather and as a predictor of conditions in the near future.

Annual, seasonal, and monthly gridded normals are provided for three time-periods:
- 100-year (1901–2000)
- 30-year (1991–2020)
- 15-year (2006–2020)

Daily gridded normals are provided for two time-periods:
- 30-year (1991–2020)
- 15-year (2006–2020)

Documentation for this dataset is available at the [Planetary Computer Data Catalog](https://planetarycomputer.microsoft.com/dataset/noaa-climate-normals-gridded).

### Data Access
This notebook works with or without an API key, but you will be given more permissive access to the data with an API key. The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) sets the environment variable "PC_SDK_SUBSCRIPTION_KEY" when your server is started. The API key may be manually set via the following code:

```python
pc.settings.set_subscription_key(<YOUR API Key>)
```

The datasets hosted by the Planetary Computer are available in [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more.

In [ ]:
import planetary_computer
import pystac_client

# Open the Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://pct-apis-staging.westeurope.cloudapp.azure.com/stac/",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("noaa-climate-normals-gridded")
collection

In [ ]:
# Search the catalog and collection for desire
search = catalog.search(
    collections=collection,
    query={
        "noaa_climate_normals:period": {
            "eq": "1991-2020"
        },  # Available periods = "1901-2000", "1991-2020", and "2006-2020"
        "noaa_climate_normals:frequency": {
            "eq": "monthly"
        },  # Available frequencies = "seasonal", "annual", "monthly", and "daily"
        "noaa_climate_normals:time_index": {
            "eq": "4"
        },  # Available time indices = seasonal (1-4), monthly (1-12), and daily (1-365)
    },
)
items = list(search.get_items())
items

### Available Assets & Metadata

Let's display the available [assets](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md#asset-object) and metadata for NOAA U.S. Gridded Climate Normals. 

In [ ]:
import rich.table

# Assets
t_assets = rich.table.Table("Key", "Value")
for key, asset in items[0].assets.items():
    t_assets.add_row(key, asset.title)
t_assets

In [ ]:
# Metadata
t_metadata = rich.table.Table("Key", "Value")
for k, v in sorted(items[0].properties.items()):
    t_metadata.add_row(k, str(v))
t_metadata

### Loading the gridded data
Now let's load STAC items into an xarray dataset using [odc-stac](https://github.com/opendatacube/odc-stac).

In [ ]:
import odc.stac

ds = odc.stac.load(items)
ds

### Displaying the gridded data

Let's display the NOAA U.S. Gridded Climate Normals.

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Mercator())

prcp = ds.prcp_norm.sel(method="nearest")

prcp.plot(
    ax=ax,
    cmap="YlGnBu",
    transform=ccrs.PlateCarree(),
)

ax.add_feature(cartopy.feature.STATES)
ax.add_feature(cfeature.COASTLINE, linewidth=1)
ax.add_feature(cfeature.OCEAN, zorder=0, linewidth=0, facecolor="#333333")
ax.add_feature(cfeature.LAND, zorder=0, linewidth=0, facecolor="#5A5A5A")
ax.set_title(
    "NOAA U.S. Gridded Climate Normals \n 1991-2020 April Precipitation Normal (inches)",
    fontweight="bold",
    fontsize="10",
)